In [1]:
import pandas as pd
import numpy as np
import json
import joblib
import os
import benchmarks
import transform_functions
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler, MaxAbsScaler, Normalizer, StandardScaler, PowerTransformer, QuantileTransformer

In [2]:
key_features = ['consumer_loans_assets', 'dep_cap_assets', 'loans_cap_assets', 'business_loans_assets', 'asset_tot_asset']
with open("data/kpi.json", "r") as kpi_json:
    kpi_list = json.load(kpi_json)
data_kpis = pd.read_csv("data/kpis.csv")
data_kpis = data_kpis[data_kpis["ind"].isin(key_features)]
kpis = data_kpis[data_kpis["ind"].isin(key_features)]["ind"].unique()
#print(kpis)

In [3]:
oscwd = os.getcwd()

# Load dataframe with average weighted value of loan segments
df_entidad = pd.read_csv(os.path.join(oscwd, "data/ENTIDAD.txt"), encoding="utf-8",
                            dtype = {
                                    'EENTIDAD': int, 'CODNUM': int, 
                                    'NOMBRE CORTO': str, "VIGENTE": int, 
                                    "FECHA_ALTA": str, "BAJA": str
                                    }
                                    # parse_dates=['Periodo'],
                            )
df_entidad = df_entidad[["EENTIDAD", "CODNUM", "NOMBRE CORTO", "VIGENTE", "FECHA_ALTA", "BAJA"]]
# Load dataframe with operative banks
active = pd.DataFrame(df_entidad["EENTIDAD"][df_entidad["VIGENTE"]==1])
active.columns = ["bank"]
# Create list of operative banks that are actually in the balance sheet statement
# banks = [a for a in active["bank"] if a in df_evoldatos["bank"].unique()]
#print(active)

In [4]:
not_operative = pd.read_csv("data/not_predicted.csv", header=0)
#not_operative.columns

In [5]:
pub_gde = [11, 14, 29, 300] # list(df_entidad["CODIGO"][df_entidad["Grupo"]==8.0])
digit = [143, 158, 384, 45030]
auto = [340, 44059, 44088, 44092, 44093, 44094, 44095, 44096, 44098, 44099]
drop_ent = pub_gde + digit + auto
#print(drop_ent)

In [6]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("final/gh_2008_2018_v2") if isfile(join("final/gh_2008_2018_v2", f))]
onlyfiles = [f for f in onlyfiles if f.startswith("Silhouette Method") and f.endswith(".csv")]
print(onlyfiles)

['Silhouette Method-KMeans-[4, 4]-MaxAbs Scaler-Isolation Forest.csv', 'Silhouette Method-KMeans-[4, 4]-MaxAbs Scaler-Local Outlier Factor.csv', 'Silhouette Method-KMeans-[4, 4]-MaxAbs Scaler-Robust Covariance.csv', 'Silhouette Method-KMeans-[4, 4]-Power Transformer-Isolation Forest.csv', 'Silhouette Method-KMeans-[4, 4]-Power Transformer-One-Class SVM (SGD).csv', 'Silhouette Method-KMeans-[4, 4]-Power Transformer-One-Class SVM.csv', 'Silhouette Method-KMeans-[4, 4]-Power Transformer-Robust Covariance.csv', 'Silhouette Method-KMeans-[4, 4]-Robust Scaler-Local Outlier Factor.csv', 'Silhouette Method-KMeans-[4, 4]-Standard Scaler-One-Class SVM (SGD).csv', 'Silhouette Method-KMeans-[4, 4]-Standard Scaler-One-Class SVM.csv', 'Silhouette Method-KMedoids-[4, 4]-Power Transformer-Isolation Forest.csv', 'Silhouette Method-KMedoids-[4, 4]-Power Transformer-One-Class SVM (SGD).csv', 'Silhouette Method-KMedoids-[4, 4]-Power Transformer-Robust Covariance.csv', 'Silhouette Method-MiniBatchKMeans-[4

In [11]:
all_groups_df = pd.DataFrame({"bank":active["bank"]})
for n, o in enumerate(onlyfiles):
    path_gh = f"final/gh_2008_2018_v2/{o}"
    split1 = path_gh.split("[")[0]
    split2 = path_gh.split("]")[-1]
    path_model = f"{split1}[]{split2}".split(".")[0]+".pkl"
    # print(o)
    #print(path_model)
    groups = pd.read_csv(path_gh)
    groups.columns = ["bank", "method"+str(n)]
    bank_list = [b for b in groups["bank"]]
    outliers = [a for a in active["bank"] if a not in bank_list]
    outliers = [a for a in outliers if a not in list(not_operative["bank"])]
    outliers = [a for a in outliers if a not in drop_ent]
    start = 2008
    end = 2018
    data = transform_functions.get_mean_pivot_table(data_kpis, 0, 2, 1, 3, start, end)
    df_corr, data = benchmarks.find_and_drop_correlated_features(data, labels_col="ent", threshold=0.7, drop=True, plot=False)
    data = data[data["ent"].isin(outliers)]
    data_arr = data.iloc[:,1:].to_numpy()
    clustering_model = joblib.load(path_model)
    print(clustering_model)
    predicted_outliers = clustering_model.predict(data_arr)
    outliers_df = pd.DataFrame({"bank":outliers, "method"+str(n):predicted_outliers})
    print(outliers_df)
    group_df = pd.concat([groups, outliers_df])

    all_groups_df = pd.merge(all_groups_df, group_df, how="left", on="bank")

print(all_groups_df)


The following columns were dropped: []
KMeans(algorithm='elkan', max_iter=1000, n_clusters=4)
    bank  method0
0     72        3
1    339        0
2  65203        1
The following columns were dropped: []
KMeans(algorithm='elkan', max_iter=1000, n_clusters=4)
     bank  method1
0      45        0
1      72        0
2     165        1
3     191        0
4     198        0
5     259        0
6     285        0
7     331        3
8     332        3
9     339        1
10  44077        3
11  44090        3
12  65203        3
The following columns were dropped: []
KMeans(algorithm='elkan', max_iter=1000, n_clusters=4)
    bank  method2
0      7        0
1     17        0
2     72        0
3    332        3
4    339        1
5  65203        3
The following columns were dropped: []
KMeans(algorithm='elkan', max_iter=1000, n_clusters=4)
   bank  method3
0   165        0
1   198        0
2   332        0
3   339        3
The following columns were dropped: []
KMeans(algorithm='elkan', max_iter=1

<Figure size 432x288 with 0 Axes>

In [12]:
all_groups_df.to_csv("final\gh_2008_2018_v2\ghs_inliers_outliers_predicted.csv", sep=",", index=False)

In [ ]:
"""groups = pd.read_csv(path_gh)
bank_list = [b for b in groups["sample"]]"""
#bank_list

In [ ]:
"""outliers = [a for a in active["bank"] if a not in bank_list]
outliers = [a for a in outliers if a not in list(not_operative["bank"])]
outliers = [a for a in outliers if a not in drop_ent]"""
#outliers

In [ ]:
"""# data_kpis = pd.read_csv("data/kpis.csv")
start = 2008
end = 2018
data = transform_functions.get_mean_pivot_table(data_kpis, 0, 2, 1, 3, start, end)
df_corr, data = benchmarks.find_and_drop_correlated_features(data, labels_col="ent", threshold=0.7, drop=True, plot=False)
data = data[data["ent"].isin(outliers)]
data_arr = data.iloc[:,1:].to_numpy()"""
#data_arr.shape

In [ ]:
"""clustering_model = joblib.load(path_model)"""
#clustering_model

In [ ]:
"""predicted_outliers = clustering_model.predict(data_arr)"""

In [ ]:
"""#print(outliers)
#print(predicted_outliers)
outliers_df = pd.DataFrame({"sample":outliers, "Silhouette Method":predicted_outliers})
#print(groups)
group_df = pd.concat([groups, outliers_df])
print(len(group_df))
display(group_df)"""